In [1]:
import pandas as pd
import numpy as np
import random

In [142]:
df_truth = pd.read_csv('truthvalue.csv', dtype=object)

df_truth = df_truth.append(df_truth,ignore_index=True)

In [ ]:
###############################################   attribute reshuffling  ###############################

In [143]:
df_truth = df_truth.drop(df_truth.columns[[9, 10, 11, 12, 13, 14, 15, 16 ]], axis=1)
df_truth['label'] = 1
df_truth_attribute_mixed = df_truth.sample(n=500, axis=0)
df_subtract = df_truth[~df_truth.isin(df_truth_attribute_mixed)].dropna()
df_truth_attribute_mixed['label'] = 0

In [144]:
for row in range(500):
    for i in range (2):
        random_cell_1 = random.randint(0,8)
        random_cell_2 = random.randint(0,8)
        if random_cell_1 != random_cell_2:
            temp = df_truth_attribute_mixed.iloc[row,random_cell_1]
            df_truth_attribute_mixed.iloc[row,random_cell_1] = df_truth_attribute_mixed.iloc[row,random_cell_2]
            df_truth_attribute_mixed.iloc[row,random_cell_2] = temp

In [159]:
df_attribute_shuffled = pd.concat([df_subtract, df_truth_attribute_mixed])
df_attribute_shuffled = df_attribute_shuffled.sample(frac=1).reset_index(drop=True)
df_attribute_shuffled.to_csv('lstm_dataset_attribute_shuffled.csv',index=False)


In [65]:
###############################################   Introduce NaN values  ###############################

In [146]:
df_truth = pd.read_csv('truthvalue.csv', dtype=object)
df_truth = df_truth.drop(df_truth.columns[[9, 10, 11, 12, 13, 14, 15, 16 ]], axis=1)
df_truth['label'] = 1
df_truth_none_introduced = df_truth.sample(n=500, axis=0)
df_subtract = df_truth[~df_truth.isin(df_truth_none_introduced)].dropna()
df_truth_none_introduced['label'] = 0

In [147]:
for row in range(500):
    random_cell_1 = random.randint(0,8)
    df_truth_none_introduced.iloc[row,random_cell_1] = np.nan

In [158]:
df_truth_none_cells = pd.concat([df_subtract, df_truth_none_introduced])
df_truth_none_cells = df_truth_none_cells.sample(frac=1).reset_index(drop=True)
df_truth_none_cells.to_csv('lstm_dataset_Nan_introduced.csv',index = False)

In [ ]:
###############################################   Introduce erroneous values  ###############################

In [149]:
df_truth = pd.read_csv('truthvalue.csv', dtype=object)
df_hospital = pd.read_csv('dirty_data_transformed.csv', dtype=object)
df_hospital = df_hospital.drop(df_hospital.columns[[3,4]], axis=1)

df_truth['label'] = 1
df_hospital['label'] = 1

df_truth = df_truth.drop(df_truth.columns[[9, 10, 11, 12, 13, 14, 15, 16 ]], axis=1)
df_hospital = df_hospital.drop(df_hospital.columns[[9, 10, 11, 12, 13, 14, 15, 16 ]], axis=1)

provider = df_truth['ProviderNumber'].tolist()
hospital = df_truth['HospitalName'].tolist()
address = df_truth['Address1'].tolist()
city = df_truth['City'].tolist()
state = df_truth['State'].tolist()
zipcode = df_truth['ZipCode'].tolist()
county = df_truth['CountyName'].tolist()
phone = df_truth['PhoneNumber'].tolist()
hospType = df_truth['HospitalType'].tolist()

combined_hosp = list(zip(provider, hospital, address, city, state, zipcode, county, phone,hospType))


In [150]:
#hospital dirty dataset; maintain dirty dataset separately
provider = df_hospital['ProviderNumber'].tolist()
hospital = df_hospital['HospitalName'].tolist()
address = df_hospital['Address1'].tolist()
city = df_hospital['City'].tolist()
state = df_hospital['State'].tolist()
zipcode = df_hospital['ZipCode'].tolist()
county = df_hospital['CountyName'].tolist()
phone = df_hospital['PhoneNumber'].tolist()
hospType = df_hospital['HospitalType'].tolist()


combined_dirty = list(zip(provider, hospital, address, city, state, zipcode, county, phone, hospType))
len(combined_dirty)


1000

In [151]:
print(combined_hosp[42])
print(combined_dirty[42])

('10001', 'SOUTHEAST ALABAMA MEDICAL CENTER', '1108 ROSS CLARK CIRCLE', 'DOTHAN', 'AL', '36302', 'HOUSTON', '3347938701', 'Acute Care Hospitals')
('10001', 'SOUTHEAST ALABAMA MEDICAL CENTER', '1108 ROSS CLARK CIRCLE', 'DOTHAN', 'xL', '36302', 'HOUSTON', '3347938701', 'Acute Care Hospitals')


In [152]:
#Calculating error or dirty cells

for i in range(len(combined_dirty)):
    for j in range(len(combined_dirty[0])):
        flag = False
        if combined_dirty[i][j] != combined_hosp[i][j]:
            flag = True
    if flag == True:    
        df_hospital.iloc[i,9] = 0

In [157]:
df_hospital = df_hospital.sample(frac=1).reset_index(drop=True)
df_hospital.to_csv('lstm_dataset_errenous.csv',index = False)

In [ ]:
################################### generating 2000 tuples for LSTM ######################

In [418]:
import math
from sklearn.utils import shuffle

df_truth = pd.read_csv('truthvalue.csv', dtype=object)
df_truth = df_truth.append(df_truth,ignore_index=True)



In [419]:

#drop few columns
df_truth = df_truth.drop(df_truth.columns[[9,10, 11, 12, 13, 14, 15, 16 ]], axis=1)
df_truth['label'] = 1

truth_rows = df_truth.shape[0]
frac = (int)(truth_rows/2)

df_false = df_truth.sample(n = frac, axis=0)

df_truth = df_truth[~df_truth.isin(df_false)].dropna()
df_false['label'] = 0


rows = df_false.shape[0]
columns = df_false.shape[1] - 1

diff = math.ceil(rows/columns)

In [420]:
list_of_df = []


def sampler(rows, columns, df):
    diff = math.ceil(rows/columns)
    print (diff)
    for i in range(columns):
        #check the remaining samples row count, if it lesser than the sample size. terminate by appending the remaining values.
        df_row_count = df.shape[0]
        if df_row_count < diff:
            list_of_df.append(df)
        else:
            df_sample = df.sample(diff, axis=0)
            list_of_df.append(df_sample)
            df = df[~df.isin(df_sample)].dropna()          
    return list_of_df



In [421]:
#sample the false dataset into different tuples groups
#each group will have one column with shuffled values

y = sampler(rows, columns, df_false)


for idx, val in enumerate(y):
    p = shuffle(y[idx].iloc[:,idx].values)
    y[idx].iloc[:,idx] = p
    
df_false = pd.concat(y)


112


In [422]:
rows = df_false.shape[0]
columns = df_false.shape[1] - 1

def introduce_NAN(percentage,rows,columns,df):
    iter = (int)((percentage/100)*rows)
    print (iter)
    for row in range(iter):
        random_cell_1 = random.randint(0,columns-1)
        df.iloc[row,random_cell_1] = np.nan
    return df



#if it is a string add a XX appended in the end.
def introduce_error(percentage,rows,columns,df):
    iter = (int)((percentage/100)*rows)
    for row in range(iter):
        random_cell_1 = random.randint(0,columns-1)
        orig = df.iloc[row,random_cell_1]
        if df.iloc[row,random_cell_1] is not None:
            if isinstance(df.iloc[row,random_cell_1], int):
                #print (df.iloc[row,random_cell_1])
                temp = int(df.iloc[row,random_cell_1]) + random.randint(1,1000)
                df.iloc[row,random_cell_1] = temp  
            elif isinstance(df.iloc[row,random_cell_1], float):
                #print (df.iloc[row,random_cell_1])
                temp = int(df.iloc[row,random_cell_1]) + random.rand(1,1000)
                df.iloc[row,random_cell_1] = temp 
            else:
                #print (df.iloc[row,random_cell_1])
                #str_len = len(str(df.iloc[row,random_cell_1]))
                #df.iloc[row,random_cell_1][random.randint(0,str_len)] = x
                df.iloc[row,random_cell_1] =  df.iloc[row,random_cell_1] + 'XX'          
        
        #print (orig,df.iloc[row,random_cell_1])
    return df
        
#input - first parameter - percentage of none values to be introduced
#second - row count, third - column count, fourth parameter  = df itself.
df_false = introduce_error(5,rows, columns, df_false)
df_false = introduce_NAN(5,rows, columns, df_false)

frames = [df_truth, df_false]

df_final = pd.concat(frames)
df_final = df_final.sample(frac=1).reset_index(drop=True)



50


In [423]:
df_final.to_csv('final.csv',index = False)